In [10]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt

In [23]:
class Summary:
    def __init__(self, underlying, triple):
        self.UNDERLYING = underlying
        self.TRIPLE = triple
        self.STARTING_VALUE_UNDER = 3000
        self.STARTING_VALUE_TRIP = -1000
        self.TICKERS = [underlying, triple]

        self.data = yf.download(self.TICKERS, period="8y", group_by="ticker", auto_adjust=False)
        self.dat_under = self.data[underlying]['Close']
        self.dat_trip = self.data[triple]['Close']

        self.shares_under = self.STARTING_VALUE_UNDER / self.dat_under[0]
        self.shares_trip = self.STARTING_VALUE_TRIP / self.dat_trip[0]
        dates = [d.strftime('%Y-%m-%d') for d in self.data.index.date]
        self.dfr = pd.DataFrame(data = {f"{underlying}_Close": self.dat_under})
        self.dfr.index = dates
        self.dfr[f"{underlying}_per_change"] = 1
        self.dfr[f"{underlying}_cumulative"] = 1
        self.dfr[f"{triple}_Close"] = self.dat_trip
        self.dfr[f"{triple}_per_change"] = 1
        self.dfr[f"{triple}_cumulative"] = 1
        self.dfr[f"ideal_per_change"] = 1
        self.dfr[f"ideal_cumulative"] = 1


    def summary_stats(self):
        for i in range(1, len(self.dat_under)):
            self.dfr[f"{self.UNDERLYING}_per_change"][i] = self.dat_under[i] / self.dat_under[i-1] - 1
            self.dfr[f"{self.TRIPLE}_per_change"][i] = self.dat_trip[i] / self.dat_trip[i-1] - 1
            self.dfr["ideal_per_change"][i] = (self.dat_under[i] / self.dat_under[i-1] - 1) *3

            self.dfr[f"{self.UNDERLYING}_cumulative"][i] = self.dfr[f"{self.UNDERLYING}_cumulative"][i-1]*(1 + self.dfr[f"{self.UNDERLYING}_per_change"][i])
            self.dfr[f"{self.TRIPLE}_cumulative"][i] = self.dfr[f"{self.TRIPLE}_cumulative"][i-1]*(1 + self.dfr[f"{self.TRIPLE}_per_change"][i])
            self.dfr["ideal_cumulative"][i] = self.dfr["ideal_cumulative"][i-1]*(1 + self.dfr["ideal_per_change"][i])


    def calc_returns(self):
        quart_under = self.data[self.UNDERLYING]['Adj Close'].resample('ME').last().pct_change()*3
        quart_trip = self.data[self.TRIPLE]['Adj Close'].resample('ME').last().pct_change()
        plt.plot(quart_under, label = f"{self.UNDERLYING} quarterly returns, beta adjusted", linewidth=0.85)
        plt.plot(quart_trip, label = f"{self.TRIPLE} quarterly returns", linewidth = 0.85)
        plt.legend()
        plt.grid(True)
        plt.show()
        quart_under = round(quart_under, 3)
        quart_trip = round(quart_trip, 3)

        d = pd.DataFrame(data = {(f"{self.UNDERLYING} quarterly returns (%)"): quart_under, f"{self.TRIPLE} quarterly returns (%)": quart_trip})
    
    
    def init_portfolio(self):
        self.dfr[f"portfolio_{self.UNDERLYING}_long"]= 1
        self.dfr[f"portfolio_{self.TRIPLE}_short"]= 0
        self.dfr["portfolio_total"]=0
        self.dfr["beta_exposure"]=0


    def calc_portfolio(self, i):      
        self.dfr[f"portfolio_{self.UNDERLYING}_long"][i] = self.shares_under * self.dfr[f"{self.UNDERLYING}_Close"][i]
        self.dfr[f"portfolio_{self.TRIPLE}_short"][i] = self.shares_trip * self.dfr[f"{self.TRIPLE}_Close"][i]
        self.dfr["portfolio_total"][i] = self.dfr[f"portfolio_{self.UNDERLYING}_long"][i] + self.dfr[f"portfolio_{self.TRIPLE}_short"][i]
        self.dfr["beta_exposure"][i] = (self.shares_under * self.dfr[f"{self.UNDERLYING}_Close"][i]) + (self.shares_trip * self.dfr[f"{self.TRIPLE}_Close"][i] * 3)
        #num shares * share price * -3 + num shares * share price
        

    def plot_portfolio(self):
        self.dfr = round(self.dfr, 2)
        self.dfr.to_csv('new.csv')
        plt.plot(self.dfr[f"portfolio_{self.UNDERLYING}_long"], label = f"portfolio_{self.UNDERLYING}_long", linewidth=0.85)
        plt.plot(self.dfr[f"portfolio_{self.TRIPLE}_short"], label = f"portfolio_{self.TRIPLE}_short", linewidth=0.85)
        plt.plot(self.dfr['portfolio_total'], label = "portfolio_total", linewidth=0.85)
        plt.plot(self.dfr['beta_exposure'], label = "beta_exposure", linewidth=0.85)
        plt.legend()
        plt.grid(True)
        plt.savefig("new.png")
        plt.show()

    def beta_norm_strategy(self):
        for i in range(1, len(self.dat_under)):
            if (self.dfr['beta_exposure'][i] > 150):
                amt = self.dfr['beta_exposure']/3/self.dfr[f"{self.TRIPLE}_Close"]
                self.shares_trip -= amt
                print(f"selling {amt} shares of SPXL")
            elif (self.dfr['beta_exposure'][i] < -150):
                amt = self.dfr['beta_exposure']/self.dfr[f"{self.UNDERLYING}_Close"]
                self.shares_under += amt
                print(f"buying {amt} shares of SPY")
            self.calc_portfolio(i)
            

            

In [24]:
spy = Summary(underlying="SPY", triple="SPXL")
spy.summary_stats()
spy.calc_returns()
spy.init_portfolio()
spy.beta_norm_strategy()
spy.plot_portfolio()

[*********************100%***********************]  2 of 2 completed
/var/folders/l9/9g52f8pd74xdlydy3kyw4rhc0000gn/T/ipykernel_3754/2942721935.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.shares_under = self.STARTING_VALUE_UNDER / self.dat_under[0]
/var/folders/l9/9g52f8pd74xdlydy3kyw4rhc0000gn/T/ipykernel_3754/2942721935.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.shares_trip = self.STARTING_VALUE_TRIP / self.dat_trip[0]
/var/folders/l9/9g52f8pd74xdlydy3kyw4rhc0000gn/T/ipykernel_3754/2942721935.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future 

<IPython.core.display.Javascript object>

/var/folders/l9/9g52f8pd74xdlydy3kyw4rhc0000gn/T/ipykernel_3754/2942721935.py:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (self.dfr['beta_exposure'][i] > 150):
/var/folders/l9/9g52f8pd74xdlydy3kyw4rhc0000gn/T/ipykernel_3754/2942721935.py:85: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (self.dfr['beta_exposure'][i] < -150):
/var/folders/l9/9g52f8pd74xdlydy3kyw4rhc0000gn/T/ipykernel_3754/2942721935.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by

<IPython.core.display.Javascript object>

In [ ]:
# -----------------------------
# Step 1: Download data
# -----------------------------

csv_file_path = 'spy.csv'
df = pd.read_csv(csv_file_path, index_col=0, parse_dates = True)

spy_df = df["SPY_Close"]
spxl_df = df["SPXL_Close"]

spy_df.to_csv("test.csv")
# beta_df = pd.read_csv("beta.csv", index_col=0, parse_dates=True)

spy_df = pd.DataFrame()
spy_df["Open"] = df["SPY_Close"]
spy_df["High"] = df["SPY_Close"]
spy_df["Low"] = df["SPY_Close"]
spy_df["Close"] = df["SPY_Close"]
spy_df["Volume"] = 1  # or use real volume if available

# Optional: set datetime index
# spy_df["Date"] = pd.to_datetime(df["Date"])
# spy_df.set_index("Date", inplace=True)


spxl_df = pd.DataFrame()
spxl_df["Open"] = df["SPXL_Close"]
spxl_df["High"] = df["SPXL_Close"]
spxl_df["Low"] = df["SPXL_Close"]
spxl_df["Close"] = df["SPXL_Close"]
spxl_df["Volume"] = 1


# spxl_df["Date"] = pd.to_datetime(df["Date"])
# spxl_df.set_index("Date", inplace=True)

# Align dates and calculate dummy beta exposure (replace with your own logic)

# Backtrader expects OHLCV columns — fill in dummy values if needed
# for col in ['Open', 'High', 'Low', 'Volume']:
#     df[col] = df['Close']
# df = df[['Open', 'High', 'Low', 'Close', 'Volume', 'beta_exposure']]

# -----------------------------
# Step 2: Custom Data Feed
# -----------------------------

# class BetaExposureFeed(bt.feeds.PandasData):
#     lines = ('beta_exposure',)
#     params = (('beta_exposure', 0),)

# -----------------------------
# Step 3: Strategy
# -----------------------------

class BetaNeutralStrategy(bt.Strategy):
    def __init__(self):
        self.spy = self.datas[0]  # SPY
        self.spxl = self.datas[1]  # SPXL
        # self.beta_series = self.datas[2]

        self.spy_beta = 1.0
        self.spxl_beta = 3.0
        self.has_bought = False
    def next(self):
        # beta_now = self.beta_series[0]
        # cash = self.broker.get_cash()
        # df = self.spy.datetime.date

        if not self.has_bought:
            a = 3000/245.52999877929700
            b = 1000/35.619998931884800
            self.buy(data=self.datas[0], size = a)
            self.sell(data=self.datas[1], size = b)
            self.has_bought = True
            print(f"{self.datas[0].datetime.date(0)} | Buying {a} shares of {self.datas[0]._name}")
            print(f"{self.datas[1].datetime.date(0)} | Selling {b} shares of {self.datas[0]._name}")

        spy_pos = self.getposition(self.datas[0]).size
        spxl_pos = self.getposition(self.datas[1]).size

        # spy_pos = self.getposition(self.spy).size
        # spxl_pos = self.getposition(self.spxl).size

        # beta_now = (spxl_pos*self.spxl[0]) * -3 + (spy_pos*self.spy[0])

        beta_now = abs((spxl_pos * self.datas[1])) * -3 + spy_pos
        



        cash = self.broker.get_cash()
        dt = self.spy.datetime.date(0)
        print(f"{dt} | Beta Exposure: {beta_now:.2f} | Cash: {cash:.2f}")

        # Get current positions

        # Calculate current beta exposure from positions
        # portfolio_beta = spy_pos * self.spy_beta + spxl_pos * -self.spxl_beta
        # print(f"Current Portfolio Beta: {portfolio_beta:.2f} | SPY: {spy_pos}, SPXL: {spxl_pos}")
        print(f"SPY: {spy_pos}, SPXL: {spxl_pos}")


        if beta_now > 1000:
            # We are too long, increase short SPXL
            spxl_price = self.spxl.close[0]
            needed_beta = -beta_now
            shares = round(needed_beta / self.spxl_beta / spxl_price)
            print(f"→ Selling short {shares} SPXL to reduce beta")
            self.sell(data=self.spxl, size=shares)

        elif beta_now < -1000:
            # We are too short, increase long SPY
            spy_price = self.spy.close[0]
            needed_beta = -beta_now
            shares = round(needed_beta / self.spy_beta / spy_price)
            print(f"→ Buying {shares} SPY to reduce beta")
            self.buy(data=self.spy, size=shares)

# -----------------------------
# Step 4: Setup and Run Cerebro
# -----------------------------

cerebro = bt.Cerebro()
cerebro.broker.set_cash(100000)
cerebro.addstrategy(BetaNeutralStrategy)

# Add SPY and SPXL price data
spy_feed = bt.feeds.PandasData(dataname=spy_df)
spxl_feed = bt.feeds.PandasData(dataname=spxl_df)
# beta_feed = BetaExposureFeed(dataname=beta_df)

cerebro.adddata(spy_feed)   # datas[0]
cerebro.adddata(spxl_feed)  # datas[1]
# cerebro.adddata(beta_feed)  # datas[2]


results = cerebro.run()
figs = cerebro.plot(style='candlestick')[0]
figures = cerebro.plot()
fig = figs[0]  # grab the first figure

fig.savefig("backtrader_plot.png", dpi=300, bbox_inches='tight')


2017-07-17 | Buying 12.21846623595943 shares of 
2017-07-17 | Selling 28.07411650719794 shares of 
2017-07-17 | Beta Exposure: 0.00 | Cash: 100000.00
SPY: 0, SPXL: 0
2017-07-18 | Beta Exposure: -2992.83 | Cash: 98000.10
SPY: 12.21846623595943, SPXL: -28.07411650719794
→ Buying 12 SPY to reduce beta
2017-07-19 | Beta Exposure: -3029.68 | Cash: 95036.22
SPY: 24.21846623595943, SPXL: -28.07411650719794
→ Buying 12 SPY to reduce beta
2017-07-20 | Beta Exposure: -3020.21 | Cash: 92071.02
SPY: 36.218466235959426, SPXL: -28.07411650719794
→ Buying 12 SPY to reduce beta
2017-07-21 | Beta Exposure: -3002.32 | Cash: 89108.46
SPY: 48.218466235959426, SPXL: -28.07411650719794
→ Buying 12 SPY to reduce beta
2017-07-24 | Beta Exposure: -2986.10 | Cash: 86146.62
SPY: 60.218466235959426, SPXL: -28.07411650719794
→ Buying 12 SPY to reduce beta
2017-07-25 | Beta Exposure: -2996.84 | Cash: 83177.58
SPY: 72.21846623595943, SPXL: -28.07411650719794
→ Buying 12 SPY to reduce beta
2017-07-26 | Beta Exposure:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>